# Import library

In [1]:
# deal with data
import pandas as pd
import numpy as np  # only used for synthetic data
import random  # only used for synthetic data
from datetime import datetime  # only used for synthetic data
import re

# load data
import glob

# deal with excel
import xlsxwriter
import os
import shutil
import win32com.client as win32

# deal with directory
from pathlib import Path

In [2]:
procss = input('Mode(initial/auto): ')
if (procss!='initial') and (procss!='auto'):
    raise ValueError('Please type a right name by hint.')

Mode(initial/auto):  auto


In [3]:
date_interval = input('week identify interval(ex.20210531_20210606): ')
if not re.match(r'\d{8}_\d{8}', date_interval):  # match 本身就會從起始位置開始匹配
    raise ValueError('Please type a right date format by hint.')

week identify interval(ex.20210531_20210606):  20210531_20210606


### 檢查每周辨識的資料夾是否存在，不存在創建一個，存在就不另行創建

In [4]:
week_identify_directory = os.path.abspath(f"../Data/week_identify/{date_interval}")

In [5]:
Path(week_identify_directory).mkdir(parents=False, exist_ok=True)

# Deal with data

In [6]:
# extract file date
def extract_date(x):
    res = re.findall(r'\d+(?=.csv)', file)
    if len(res)==1:
        res=res[0]
    else:
        raise ValueError('extract date error')
    return res

In [7]:
# run several date
deed_detail = pd.DataFrame()
date_li = []
for file in glob.glob(f"../Data/week_identify/{date_interval}/file*.csv"):
    print(f"load file: {file}")
    date_li.append(extract_date(file))
    df = pd.read_csv(file)
    deed_detail = pd.concat([deed_detail, df])
    print(f"all data len: {len(deed_detail)}")
deed_detail = deed_detail.reset_index(drop=True)

load file: ../Data/week_identify/20210531_20210606\file_20210605.csv
all data len: 863
load file: ../Data/week_identify/20210531_20210606\file_20210606.csv
all data len: 1403


In [8]:
deed_detail.head()

,date,identify_date,store_id,deed_type,file_name,identify_status
0,2021/6/5,2021/6/6,Store_CA088_Eric,A,pic1.jpg,error1
1,2021/6/5,2021/6/6,Store_CA133_Eric,A,pic2.jpg,error1
2,2021/6/5,2021/6/6,Store_DC140_Eric,A,pic3.jpg,error1
3,2021/6/5,2021/6/6,Store_XC034_Eric,U,pic4.jpg,error1
4,2021/6/5,2021/6/6,Store_AB121_Eric,A,pic5.jpg,error2


In [9]:
# 日期排序
date_li.sort()

In [10]:
date_li

['20210605', '20210606']

In [11]:
date_interval_str = date_li[0] + '_' + date_li[-1]

In [12]:
franchise_store = pd.read_excel('../Data/store_data.xlsx', engine='openpyxl')

In [13]:
franchise_store

,品牌,店代碼,店名稱
0,Brand_TCE,Store_MB001_Eric,店A1
1,Brand_TCE,Store_MB002_Eric,店A2
2,Brand_TCE,Store_MB003_Eric,店A3
3,Brand_TCE,Store_MB004_Eric,店A4
4,Brand_TCE,Store_MB005_Eric,店A5
...,...,...,...
2044,Brand_YCE,Store_DD010_Eric,店U45
2045,Brand_YCE,Store_DD011_Eric,店U46
2046,Brand_YCE,Store_DD012_Eric,店U47
2047,Brand_YCE,Store_DD013_Eric,店U48


In [14]:
df = deed_detail.merge(franchise_store, how = 'left', left_on='store_id', right_on='店代碼')

In [15]:
df.head(3)

,date,identify_date,store_id,deed_type,file_name,identify_status,品牌,店代碼,店名稱
0,2021/6/5,2021/6/6,Store_CA088_Eric,A,pic1.jpg,error1,Brand_YCE,Store_CA088_Eric,店R6
1,2021/6/5,2021/6/6,Store_CA133_Eric,A,pic2.jpg,error1,Brand_YCE,Store_CA133_Eric,店R49
2,2021/6/5,2021/6/6,Store_DC140_Eric,A,pic3.jpg,error1,Brand_YCE,Store_DC140_Eric,店U10


In [16]:
print('7天內資料總數: ',len(df))

7天內資料總數:  1403


In [17]:
df = df[['date', 'identify_date', 'store_id', '品牌', '店名稱', 'deed_type', 'file_name', 'identify_status']]

In [18]:
# remove file_name is null
df = df.query("file_name==file_name")
df = df.reset_index(drop=True)

In [19]:
column_has_null_value = df.isna().any()
if column_has_null_value.any():
    raise ValueError('dataframe ha some non value')

In [20]:
def convert_error(x):
    if re.search(r'^error', x):
        res = 'error'
    else:
        res = x
    return res

In [21]:
df['identify_status_type'] = df['identify_status'].apply(convert_error)

In [22]:
df

,date,identify_date,store_id,品牌,店名稱,deed_type,file_name,identify_status,identify_status_type
0,2021/6/5,2021/6/6,Store_CA088_Eric,Brand_YCE,店R6,A,pic1.jpg,error1,error
1,2021/6/5,2021/6/6,Store_CA133_Eric,Brand_YCE,店R49,A,pic2.jpg,error1,error
2,2021/6/5,2021/6/6,Store_DC140_Eric,Brand_YCE,店U10,A,pic3.jpg,error1,error
3,2021/6/5,2021/6/6,Store_XC034_Eric,Brand_UTE,店L94,U,pic4.jpg,error1,error
4,2021/6/5,2021/6/6,Store_AB121_Eric,Brand_YCE,店N49,A,pic5.jpg,error2,error
...,...,...,...,...,...,...,...,...,...
858,2021/6/5,2021/6/6,Store_WD007_Eric,Brand_UTE,店L30,H,pic859.jpg,ok,ok
859,2021/6/5,2021/6/6,Store_WD007_Eric,Brand_UTE,店L30,U,pic860.jpg,ok,ok
860,2021/6/5,2021/6/6,Store_WD007_Eric,Brand_UTE,店L30,U,pic861.jpg,ok,ok
861,2021/6/5,2021/6/6,Store_WD007_Eric,Brand_UTE,店L30,U,pic862.jpg,ok,ok


初始自動化程序<br>
步驟如下：
1. 透過 pd.ExcelWriter，將上面跑的資料作為樣板，建一個 xlsx file 在 meta_data 資料夾當中
2. 程式會自動打開步驟一建立的檔案，然後請手動建立 Excel 巨集(巨集名稱要跟你在notebook中一樣)，完成後輸入Y，程式會先檢查你是否真的建立一個含有巨集的 Excel(副檔名為 xlsm)
3. 找到此腳本(vba_extract.py) 在你電腦的位置，我的電腦是在 D:\Anaconda3\Scripts 當中，將當前目錄更改到 D:\Anaconda3\Scripts
4. 透過 D:\Anaconda3\Scripts 當中的 vba_extract.py ，從含有巨集的 Excel 當中萃取出巨集檔 (vbaProject.bin)在當前目錄(D:\Anaconda3\Scripts)
    * 將 vbaProject.bin 改成你喜歡的名子，並移動到此 notebook 所處的資料夾
    * 將當前目錄更改回此 notebook 所處的資料夾

In [23]:
excel_macro_template_name = 'excel_macro_template'

In [24]:
Code_abspath = os.getcwd()
meta_data_abspath = os.path.abspath(os.path.join(os.getcwd(), ".."))+'\\Data\\meta_data'
Output_abspath = os.path.abspath(os.path.join(os.getcwd(), ".."))+'\\Output'
macro_name = input('Type excel macro name(ex.create_pvtb_by_week): ')

Type excel macro name(ex.create_pvtb_by_week):  create_pvtb_by_week


In [25]:
if procss=='initial':
    print('Initial mode\n=================================================')
    
    ### Step 1
    xlsx_macro_template_path = os.path.abspath(f'../Data/meta_data/{excel_macro_template_name}.xlsx')
    writer = pd.ExcelWriter(xlsx_macro_template_path)
    df.to_excel(writer, index=False, sheet_name=f'{date_interval_str}下載辨識結果', header=True)
    writer.save()

    ### Step 2
    xl= win32.Dispatch("Excel.Application")
    wb = xl.Workbooks.Open(xlsx_macro_template_path)  # , ReadOnly=1
    xl.Visible = True
    print('1. please record a macro in Excel\n2. Save as xlsm file in the same path')
    complete_step = input('Whether the above step is complete?(Y/N)')

    if complete_step == 'Y' or complete_step == 'y':
        # check excel which has macro whether exist
        if os.path.exists(f'../Data/meta_data/{excel_macro_template_name}.xlsm'):
            pass
        else:
            raise FileNotFoundError("You didn't create a macro template")
    else:
        raise ValueError('Step not completed. Please restart kernel and run notebook again')


    ### Step 3
    #### Extract a macro from Excel
    if complete_step == 'Y' or complete_step == 'y':
        file_path = 'D:\Anaconda3\Scripts'
        os.chdir(file_path)

        run_script = f"python vba_extract.py {meta_data_abspath + '/' + excel_macro_template_name + '.xlsm'}"
        os.system(run_script)

    #     meta_data_file_path = 'D:\python\deal_with_excel\Data\meta_data'
        os.rename('vbaProject.bin', meta_data_abspath+'\\'+ macro_name+'.bin')

    #     code_file_path = 'D:\python\deal_with_excel\Code'
        os.chdir(Code_abspath)
    else:
        raise ValueError('Step not completed. Please restart kernel and run notebook again')
else:
    print('Auto mode\n=================================================')

Auto mode


---
初始自動化程序<br>
步驟如下：
1. 建立一個 excel，將上方所得的 data frame 寫入 xlsx file，然後修改副檔名成 xlsm，在 load excel 巨集檔(vbaProject.bin)到 excel 中，並儲存
2. 透過 win32 套件連接 excel 應用程式
3. 打開上一步驟儲存的應用程式
4. 運行巨集
5. 儲存結果到 Output 資料夾
6. 關閉 excel 應用程式

### Step 1

In [26]:
os.path.abspath(meta_data_abspath + f'\\{excel_macro_template_name}.xlsm')

'D:\\python\\excel_auto_pivot_table-main\\Data\\meta_data\\excel_macro_template.xlsm'

In [27]:
writer = pd.ExcelWriter(meta_data_abspath + f'\\{excel_macro_template_name}.xlsx')
df.to_excel(writer, index=False, sheet_name = f'{date_interval_str}下載辨識結果', header=True)  # 這裡會覆蓋原本是 template 的資料

wb = writer.book
wb.filename = os.path.abspath(meta_data_abspath + f'\\{excel_macro_template_name}.xlsm')
if os.path.exists(meta_data_abspath + f'\\{macro_name}.bin'):
    wb.add_vba_project(os.path.abspath(meta_data_abspath + f'\\{macro_name}.bin'))
else:
    raise ValueError("Can't found macro file")
writer.save()

### Step2-Step6

In [28]:
Output_abspath

'D:\\python\\excel_auto_pivot_table-main\\Output'

In [29]:
if os.path.exists(meta_data_abspath+f'\\{excel_macro_template_name}.xlsm'):
    try:
        xl= win32.Dispatch("Excel.Application")
        wb = xl.Workbooks.Open(os.path.abspath(meta_data_abspath + f"\\{excel_macro_template_name}.xlsm"))  # , ReadOnly=1
        xl.Application.Run(f"{excel_macro_template_name}.xlsm!Module1.{macro_name}")
        # if you want to save then uncomment this line and change delete the ", ReadOnly=1" part from the open function.
        xl.Visible = True
        wb.SaveAs(os.path.abspath(Output_abspath + f"\\{date_interval_str}下載辨識結果.xlsm"))
    finally:    
        xl.Application.Quit() # Comment this out if your excel script closes

    del xl

In [30]:
# os.remove(f'../Data/meta_data/{excel_macro_template_name}.xlsm')

In [31]:
# # convert notebook.ipynb to a .py file
# !jupytext --to py auto_excel.ipynb